In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [2]:
df = pd.read_csv('PremierLeague.csv')

In [3]:
df.head()

,round_number,league_name,name,starting_at,home_team_name,away_team_name,home_team_goals,away_team_goals
0,1,Premier League,Burnley vs Manchester City,8/11/2023,Burnley,Manchester City,0,3
1,1,Premier League,Arsenal vs Nottingham Forest,8/12/2023,Arsenal,Nottingham Forest,2,1
2,1,Premier League,AFC Bournemouth vs West Ham United,8/12/2023,AFC Bournemouth,West Ham United,1,1
3,1,Premier League,Brighton & Hove Albion vs Luton Town,8/12/2023,Brighton & Hove Albion,Luton Town,4,1
4,1,Premier League,Everton vs Fulham,8/12/2023,Everton,Fulham,0,1


In [4]:
df[['starting_at']] = df[['starting_at']].apply(pd.to_datetime) 

In [5]:
def define_winner(r):
    if r['home_team_goals'] > r['away_team_goals']:
        return "home_winner"
    elif r['home_team_goals'] < r['away_team_goals']:
        return "away_winner"
    else:
        return "draw"
df['winner'] = df.apply(define_winner,axis=1)
df

,round_number,league_name,name,starting_at,home_team_name,away_team_name,home_team_goals,away_team_goals,winner
0,1,Premier League,Burnley vs Manchester City,2023-08-11,Burnley,Manchester City,0,3,away_winner
1,1,Premier League,Arsenal vs Nottingham Forest,2023-08-12,Arsenal,Nottingham Forest,2,1,home_winner
2,1,Premier League,AFC Bournemouth vs West Ham United,2023-08-12,AFC Bournemouth,West Ham United,1,1,draw
3,1,Premier League,Brighton & Hove Albion vs Luton Town,2023-08-12,Brighton & Hove Albion,Luton Town,4,1,home_winner
4,1,Premier League,Everton vs Fulham,2023-08-12,Everton,Fulham,0,1,away_winner
...,...,...,...,...,...,...,...,...,...
375,38,Premier League,Crystal Palace vs Aston Villa,2024-05-19,Crystal Palace,Aston Villa,5,0,home_winner
376,38,Premier League,Liverpool vs Wolverhampton Wanderers,2024-05-19,Liverpool,Wolverhampton Wanderers,2,0,home_winner
377,38,Premier League,Luton Town vs Fulham,2024-05-19,Luton Town,Fulham,2,4,away_winner
378,38,Premier League,Manchester City vs West Ham United,2024-05-19,Manchester City,West Ham United,3,1,home_winner


In [6]:
d = {}

for team in df.home_team_name.unique():
    df_home = df[df['home_team_name'] == team]
    df_away = df[df['away_team_name'] == team]
    
    counts_home = df_home.value_counts('winner')
    counts_away = df_away.value_counts('winner')
    
    home_winner = 0 if 'home_winner' not in counts_home else counts_home['home_winner']
    away_winner = 0 if 'away_winner' not in counts_away else counts_away['away_winner']
    
    home_draw = 0 if 'draw' not in counts_home else counts_home['draw']
    away_draw = 0 if 'draw' not in counts_away else counts_away['draw']
    
    win_away = 0 if 'away_winner' not in counts_away else counts_away['away_winner']
    lost_away = 0 if 'home_winner' not in counts_away else counts_away['home_winner']
    
    GF = df_home.home_team_goals.sum() + df_away.away_team_goals.sum()
    GA = df_home.away_team_goals.sum() + df_away.away_team_goals.sum()
    
    GD = GF - GA
    
    d[team] = {
        'Played' : home_winner + win_away + away_winner + lost_away + home_draw + away_draw,
        'Won' : home_winner + win_away,
        'Drawn' : home_draw + away_draw,
        'Lost' : away_winner + lost_away,
        'GF' : GF,
        'GD' : GD,
        'Points' : (home_winner + win_away)*3 + (home_draw + away_draw)
    }

In [7]:
def highlight_last_n_rows(n=3):
    def highlight(row):
        return ['background-color: #ee5d5d' if row.name >= 17 else '' for _ in row]
    
    return highlight

def highlight_champion():
    def highlight(row):
        # Check if the row index is the first row
        return ['background-color: gold' if row.name == 0 else '' for _ in row]
    
    return highlight

In [10]:
df_ranked = pd.DataFrame(d)
df_ranked = df_ranked.T
df_ranked = df_ranked.sort_values("Points", ascending=False).reset_index()

# Applying stylish
df_rank = (df_ranked.style
                  .apply(highlight_last_n_rows(), axis=1)
                  .apply(highlight_champion(), axis=1))
df_rank

,index,Played,Won,Drawn,Lost,GF,GD,Points
0,Manchester City,52,28,7,17,96,35,91
1,Arsenal,49,28,5,16,91,32,89
2,Liverpool,46,24,10,12,86,32,82
3,Aston Villa,43,20,8,15,76,20,68
4,Tottenham Hotspur,39,20,6,13,74,11,66
5,Chelsea,41,18,9,14,77,18,63
6,Newcastle United,41,18,6,17,85,27,60
7,Manchester United,40,18,6,16,57,3,60
8,West Ham United,41,14,10,17,60,3,52
9,Crystal Palace,36,13,10,13,57,11,49
